In [1]:
import pandas as pd

In [2]:
from dgutils.pandas import add_column

/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
sns.set(color_codes=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='ggplot')

/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
from scipy.stats import pearsonr

In [5]:
df_pred = pd.read_csv('data/for_comparison/costonzo_2009_pvalue005_prediction_growth', sep='\t',
                     header=None, names=['g1', 'g2', 'pred'])

In [6]:
# raw testing data from their github
df_raw = pd.read_csv('data/training/GO_0006281_testing_data_2016_0.txt', 
                    header=None, names=['foo', 'g1', 'g2', 'fitness'], sep=' ')[['g1', 'g2', 'fitness']]

In [7]:
# make gene id pair a sorted tuple, for joining
df_pred = add_column(df_pred, 'gene_pair', ['g1', 'g2'], lambda x, y: tuple(sorted([x, y])))
df_raw = add_column(df_raw, 'gene_pair', ['g1', 'g2'], lambda x, y: tuple(sorted([x, y])))
df_pred.drop(columns=['g1', 'g2'], inplace=True)
df_raw.drop(columns=['g1', 'g2'], inplace=True)

In [8]:
# combine duplicates
# take median for now
df_raw = df_raw.groupby(by='gene_pair', as_index=False).agg('median')

In [9]:
# intersection of the two data 
# (since we didn't load everything: raw is only a subset, pred is p<0.05 only)
print(len(df_raw), len(df_pred))
df = pd.merge(df_raw, df_pred, on='gene_pair', how='inner')
print(len(df))

2868 413400
336


In [10]:
df.head()

,gene_pair,fitness,pred
0,"(YAL015C, YER162C)",-0.0244,0.593638
1,"(YAL019W, YER162C)",-0.0021,0.765771
2,"(YAL019W, YGL175C)",-0.1066,0.688819
3,"(YAL019W, YGL244W)",0.0461,1.103655
4,"(YAL019W, YJR066W)",0.0565,0.929328


In [11]:
pearsonr(df['fitness'], df['pred'])

(0.16516683829808237, 0.002388174141878809)

In [12]:
# do not plot if we have too many data points!
if len(df) < 10000:
    df.iplot(kind='scatter', mode='markers',
            x='fitness', y='pred', size=2)